In [23]:
import getpass
import json

import requests
import sqlalchemy as sa

from sqlalchemy.orm import declarative_base, sessionmaker

In [12]:
Base = declarative_base()


class Article(Base):
    __tablename__ = 'article'

    id = sa.Column(sa.Integer, primary_key=True)
    index = sa.Column(sa.Integer)
    href = sa.Column(sa.String(256))
    title = sa.Column(sa.String(256))
    description = sa.Column(sa.String(1024))
    image = sa.Column(sa.String(512))
    tags = sa.Column(sa.String(512))
    groups = sa.Column(sa.String(512))
    author = sa.Column(sa.String(64))
    date = sa.Column(sa.DateTime)
    content = sa.Column(sa.String(None))

    def __repr__(self):
        return (
            f'Article(id={self.id!r}, href={self.href!r}, title={self.title!r}'
            f', description={self.description[:30]!r}, image={self.image!r}'
            f', tags={self.tags!r}, groups={self.groups!r}'
            f', author={self.author!r}, date={self.date!r}'
            f', content={self.content[:30]!r}'
        )

In [57]:
class ArticleEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, Article):
            res = {key: val for key, val in vars(obj).items() if not key.startswith('_')}
            res['date'] = res['date'].isoformat()
            return res

In [71]:
your_password_here = getpass.getpass()
odbc_conn_str = (
    'Driver={ODBC Driver 17 for SQL Server};'
    'Server=tcp:paper-scope-dbserver.database.windows.net,1433;'
    'Database=paper-scope-db;'
    'Uid=brian;'
    f'Pwd={your_password_here};'
    'Encrypt=yes;'
    'TrustServerCertificate=no;'
    'Connection Timeout=30;'
)

In [72]:
engine = sa.create_engine(f'mssql+pyodbc:///?odbc_connect={odbc_conn_str}')
Session = sessionmaker(engine)

In [16]:
with Session() as session:
    articles = session.query(Article).all()

In [59]:
headers = {'Content-Type': 'application/json'}
params = {'refresh': 'true'}
url = 'http://clunacy-burst.eastus.cloudapp.azure.com:9200/articles/_bulk'
idx = 0
while idx < len(articles):
    idx_max = min(idx + 500, len(articles))
    print(f'Indexing articles {idx} through {idx_max}...')
    with open('elastic_articles.json', 'w') as jsonf:
        for article in articles[idx:idx_max]:
            index = {'index': {'_id': article.id}}
            json.dump(index, jsonf)
            jsonf.write('\n')
            json.dump(article, jsonf, cls=ArticleEncoder)
            jsonf.write('\n')
    with open('elastic_articles.json', 'rb') as data:
        r = requests.post(url, headers=headers, params=params, data=data)
    if r.status_code != 200:
        break
    idx += 500

Indexing articles 0 through 500...
Indexing articles 500 through 1000...
Indexing articles 1000 through 1500...
Indexing articles 1500 through 2000...
Indexing articles 2000 through 2500...
Indexing articles 2500 through 3000...
Indexing articles 3000 through 3500...
Indexing articles 3500 through 4000...
Indexing articles 4000 through 4500...
Indexing articles 4500 through 5000...
Indexing articles 5000 through 5500...
Indexing articles 5500 through 6000...
Indexing articles 6000 through 6500...
Indexing articles 6500 through 7000...
Indexing articles 7000 through 7500...
Indexing articles 7500 through 8000...
Indexing articles 8000 through 8500...
Indexing articles 8500 through 9000...
Indexing articles 9000 through 9500...
Indexing articles 9500 through 10000...
Indexing articles 10000 through 10500...
Indexing articles 10500 through 11000...
Indexing articles 11000 through 11500...
Indexing articles 11500 through 12000...
Indexing articles 12000 through 12500...
Indexing articles 1

In [76]:
with Session() as session:
    qr = session.query(Article).filter(sa.or_(
        Article.title.contains('Anthony Edwards'),
        Article.description.contains('Anthony Edwards'),
        Article.content.contains('Anthony Edwards'),
    )).limit(50).all()

In [75]:
qr[0].href

'https://www.theringer.com/2017/5/25/16046880/miracle-mile-nuclear-movies-romance-reagan-era-80e14461144a'

In [81]:
headers = {'Content-Type': 'application/json'}
url = 'http://clunacy-burst.eastus.cloudapp.azure.com:9200/articles/_search'
data = {
    'size': 50,
    'query': {
        'multi_match': {
            'query': 'Anthony Edwards',
            'type': 'phrase',
            'fields': ['title', 'description', 'content'],
        },
    },
}
r = requests.get(url, headers=headers, data=json.dumps(data))

In [ ]:
print(r.status_code)
r.json()